In [ ]:
from IPython.display import Image
from dotenv import load_dotenv
from openai import OpenAI

# Load .env
load_dotenv(override=True)

# Configure API
client = OpenAI()
chat_model = "gpt-4o-mini"
image_model = "dall-e-3"


In [ ]:
# Write an intruction
instructions = """
You are a helpful assistant who can answer questions and generate
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "create_answer",
            "description": "Create an answer based on the question",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "create_image",
            "description": "Create an image based on the question",
        },
    },
]

def create_answer(messages: list):
    response = client.chat.completions.create(messages=messages, model=chat_model)
    return {"role": "assistant", "content": response.choices[0].message.content}


def create_image(messages: list):
    prompt = messages[-1]["content"]
    result = client.images.generate(model=image_model, prompt=prompt, n=1, style="natural")
    revised_prompt = result.data[0].revised_prompt
    print(f"revised_prompt: {revised_prompt}")
    image_url = result.data[0].url
    img = Image(url=image_url)
    return {"role": "assistant", "content": revised_prompt, "img": img}


def execute_tool(messages: list, tool_name: str):
    """Helper function to execute tool by name."""
    if tool_name == "create_answer":
        return create_answer(messages)
    elif tool_name == "create_image":
        return create_image(messages)
    return None


def chat(question: str):
    print("user:", question)

    # Get tool_calls from question
    messages = [{"role": "system", "content": instructions}] + [{"role": "user", "content": question}]
    response = client.chat.completions.create(
        messages=messages, tools=tools, model=chat_model, tool_choice="required",
    )
    tool_calls = response.choices[0].message.tool_calls

    # Create output from tool_calls
    output = []
    for tool_call in tool_calls:
        print(f"tool_call: {tool_call.function.name}()")
        result = execute_tool(messages=messages, tool_name=tool_call.function.name)
        output.append(result)

    # Print output
    for msg in output:
        print(f"{msg['role']}:\n'{msg['content']}'")
        if "img" in msg:
            display(msg["img"])


In [ ]:
question = """Write a haiku about a green dog named Sam"""
chat(question)


In [ ]:
question = """Create an image of a green dog"""
chat(question)
